In [1]:
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import csv

def read_csv():
    with open("power_generation_processed.csv") as csvfile:  # same location as data3.csv
        datareader = csv.reader(csvfile, delimiter=',')
        times = []
        categories = []
        num_categories = len(next(datareader)) - 1  # calculate number of columns
        csvfile.seek(0)  # reset dataset after the next(...) function call

        for i in range(num_categories):
            categories.append([])
        for row in datareader:
            times.append((row[0]))
            for i in range(num_categories):
                try:
                    categories[i].append(float((row[i + 1])))
                except:
                    categories[i].append(-1)

        returned_values = []
        returned_values.append(times)
        for i in range(num_categories):
            returned_values.append(categories[i])

    return returned_values

values = read_csv()
timestamps = values[0]
del timestamps[0]
geothermal_values = values[1]
del geothermal_values[0]
biomass_values = values[2]
del biomass_values[0]
biogas_values = values[3]
del biogas_values[0]
small_hydro_values = values[4]
del small_hydro_values[0]
wind_total_values = values[5]
del wind_total_values[0]
solar_pv_values = values[6]
del solar_pv_values[0]
solar_thermal_values = values[7]
del solar_thermal_values[0]
renewable_values = values[8]
del renewable_values[0]
nuclear_values = values[9]
del nuclear_values[0]
thermal_values = values[10]
del thermal_values[0]
imports_values = values[11]
del imports_values[0]
hydro_values = values[12]
del hydro_values[0]

def select_last_year(x):
    index = 0
    for i in range(len(x)):
        if x[i] == "2021.01.01.01":
            index = i
    return index

new_year_index = select_last_year(timestamps)

def delete_old_data(x, index):
    for i in range(index):
        del x[0]
    return x

geothermal_values = delete_old_data(geothermal_values, new_year_index)
biomass_values = delete_old_data(biomass_values, new_year_index)
biogas_values = delete_old_data(biogas_values, new_year_index)
small_hydro_values = delete_old_data(small_hydro_values, new_year_index)
wind_total_values = delete_old_data(wind_total_values, new_year_index)
solar_pv_values = delete_old_data(solar_pv_values, new_year_index)
solar_thermal_values = delete_old_data(solar_thermal_values, new_year_index)
renewable_values = delete_old_data(renewable_values, new_year_index)
nuclear_values = delete_old_data(nuclear_values, new_year_index)
thermal_values = delete_old_data(thermal_values, new_year_index)
imports_values = delete_old_data(imports_values, new_year_index)
hydro_values = delete_old_data(hydro_values, new_year_index)

def best_arima_parameters(ts, ratio):
    train = []
    test = []
    train_len = round(ratio * len(ts))
    test_len = len(ts) - train_len
    for i in range(train_len):
        train.append(ts[i])
    for i in range(test_len):
        test.append(ts[train_len + i])

    lowest_val_loss = np.inf  # set the lowest_validation_loss to +infinity in the beginning -S
    best_params = []
    for p in range(3):
        for d in range(3):
            for q in range(3):
                model = ARIMA(train, order=(p, d, q))
                fit_success = []
                try:
                    model_fit = model.fit()
                    pred = model_fit.predict(train_len, train_len + test_len - 1)
                #    print('ARIMA successfully fit for p=' + str(p) + ',d=' + str(d) + ',q=' + str(q) + '.')
                    fit_success = 1
                except:
               #     print('ARIMA could not fit for p=' + str(p) + ',d=' + str(d) + ',q=' + str(q) + '.')
                    fit_success = 0
                cur_val_loss = np.sum((pred - test) ** 2)  # validation loss calculation in 1 line -S
                if cur_val_loss < lowest_val_loss:
                    lowest_val_loss = cur_val_loss
                    best_params = (p, d, q)
    return best_params, lowest_val_loss

final_list = []

geothermal_list = []
geothermal_list.insert(0, "Geothermal")
output = best_arima_parameters(geothermal_values, 0.6)
geothermal_list.insert(1, output)

biomass_list = []
biomass_list.insert(0, "Biomass")
output = best_arima_parameters(biomass_values, 0.6)
biomass_list.insert(1, output)

biogas_list = []
biogas_list.insert(0, "Biogas")
output = best_arima_parameters(biogas_values, 0.6)
biogas_list.insert(1, output)

small_hydro_list = []
small_hydro_list.insert(0, "Small Hydro")
output = best_arima_parameters(small_hydro_values, 0.6)
small_hydro_list.insert(1, output)

wind_total_list = []
wind_total_list.insert(0, "Wind Total")
output = best_arima_parameters(wind_total_values, 0.6)
wind_total_list.insert(1, output)

solar_pv_list = []
solar_pv_list.insert(0, "Solar PV")
output = best_arima_parameters(solar_pv_values, 0.6)
solar_pv_list.insert(1, output)

solar_thermal_list = []
solar_thermal_list.insert(0, "Solar Thermal")
output = best_arima_parameters(solar_thermal_values, 0.6)
solar_thermal_list.insert(1, output)

renewable_list = []
renewable_list.insert(0, "Renewable")
output = best_arima_parameters(renewable_values, 0.6)
renewable_list.insert(1, output)

nuclear_list = []
nuclear_list.insert(0, "Nuclear")
output = best_arima_parameters(nuclear_values, 0.6)
nuclear_list.insert(1, output)

thermal_list = []
thermal_list.insert(0, "Thermal")
output = best_arima_parameters(thermal_values, 0.6)
thermal_list.insert(1, output)

imports_list = []
imports_list.insert(0, "Imports")
output = best_arima_parameters(imports_values, 0.6)
imports_list.insert(1, output)

hydro_list = []
hydro_list.insert(0, "Hydro")
output = best_arima_parameters(hydro_values, 0.6)
hydro_list.insert(1, output)

final_list.append(geothermal_list)
final_list.append(biomass_list)
final_list.append(biogas_list)
final_list.append(small_hydro_list)
final_list.append(wind_total_list)
final_list.append(solar_pv_list)
final_list.append(solar_thermal_list)
final_list.append(renewable_list)
final_list.append(nuclear_list)
final_list.append(thermal_list)
final_list.append(imports_list)
final_list.append(hydro_list)



print(final_list)
np.savetxt('arima_parameters.csv', final_list, fmt="%s", delimiter=',')

C:\Users\shour\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\shour\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\shour\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\shour\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings

[['Geothermal', ((0, 0, 2), 10812530.209295109)], ['Biomass', ((0, 2, 1), 1511232.6111396607)], ['Biogas', ((2, 0, 0), 531367.9509081607)], ['Small Hydro', ((2, 1, 2), 2574600.146935995)], ['Wind Total', ((0, 1, 0), 3090609714.0)], ['Solar PV', ((2, 1, 2), 48649305817.828384)], ['Solar Thermal', ((2, 0, 2), 37285217.177330494)], ['Renewable', ((2, 1, 2), 41571628930.35678)], ['Nuclear', ((2, 2, 2), 64458306.029866844)], ['Thermal', ((1, 1, 0), 36997415259.71272)], ['Imports', ((0, 1, 0), 16280550792.0)], ['Hydro', ((2, 2, 2), 1681959848.0284452)]]


C:\Users\shour\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\npyio.py:1378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)
